In [ ]:
## Полезная библиотека для решения
%pip install HeapDict
import heapdict
import numpy as np

In [ ]:
def heuristic_misplaced(state):
    """
    Количество неправильно расположенных костяшек.
    Пустое пространство не является костяшкой и не должно
    не учитывается при подсчете неправильно размещенных костяшек.

    :param state: Tuple, представляющий текущее состояние игры.
    :return: Количество неправильно расположенных костяшек.
    """

    return sum([(elem != (i + 1)) and (elem != 0) for i, elem in enumerate(state)])

In [ ]:
def test_heuristic_misplaced():
    res = []
    test_cases = [
    ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
    ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
    ((1, 2, 3, 4, 5, 6, 7, 0, 8), 1),
    ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
    ((1, 2, 3, 4, 0, 5, 7, 8, 6), 2),
]



    for i, (state, expected_value) in enumerate(test_cases):
        actual_value = heuristic_misplaced(state)
        assert actual_value == expected_value, f"Тест для расположения {state} завершился ошибкой: " \
                                               f"функия heuristic_misplaced вернула значение {actual_value}, " \
                                               f"ожидаемое значение: {expected_value}"

    print("Все тесты для heuristic_misplaced завершились успешно!")

test_heuristic_misplaced()

Все тесты для heuristic_misplaced завершились успешно!


In [ ]:
def get_pos(i):
    return np.array([i // 3, i % 3])  # To hava pos + pos functionality

def get_index(pos):
    return pos[0] * 3 + pos[1]

def manhattan_distance(i, j):
    return abs(i[0] - j[0]) + abs(i[1] - j[1])

def heuristic_manhattan(state):
    """
    Сумма манхэттенских расстояний от неправильно
    расположенных костяшек до их правильного положения.

    :param state: Tuple, представляющий текущее состояние игры.
    :return: Сумма манхэттенских расстояний.
    """

    ans = 0
    for i, elem in enumerate(state):
        if elem == 0:
            continue

        ans += manhattan_distance(get_pos(i), get_pos(elem - 1))

    return ans

In [ ]:
def test_heuristic_manhattan():
    res = []
    test_cases = [
        ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
        ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
        ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
        ((1, 2, 3, 4, 6, 0, 7, 5, 8), 3),
        ((1, 3, 0, 4, 2, 5, 7, 8, 6), 4)
    ]

    for state, expected_value in test_cases:
        actual_value = heuristic_manhattan(state)
        assert actual_value == expected_value, f"Тест для расположения {state} завершился ошибкой: " \
                                               f"функия heuristic_manhattan вернула значение {actual_value}, " \
                                               f"ожидаемое значение: {expected_value}"

    print("Все тесты для heuristic_manhattan завершились успешно!")

test_heuristic_manhattan()

Все тесты для heuristic_manhattan завершились успешно!


In [ ]:
def astar(init_state, heuristic):
    """
    A^* implementation.

    :param init_state: Tuple, представляющий текущее состояние игры.
    :param heuristic: Функция эвристики
    :return: Tuple, в котором:
        Первый элемент - это строка, представляющая оптимальный путь.
            Используйте символы 'r', 'l', 'u' и 'd' для обозначения
            'right', 'left', 'up' и 'down' соответственно.
        Второй элемент - это список, содержащий исследованные (expanded)
        алгоритмом состояния в порядке их прохождения.
    """

    moves = {
        "r": [0, 1],
        "u": [-1, 0],
        "l": [0, -1],
        "d": [1, 0]
    }

    # Test the puzzle for the ability to be solved
    inversions = 0
    for i, elem in enumerate(init_state):
        inversions += sum([(init_state[j] != 0) and (elem > init_state[j]) for j in range(i + 1, len(init_state))])

    if inversions % 2 != 0:
        # print("Нерешаемый пазл")
        return None, []

    steps = ""
    states_visited = []
    state_queue = heapdict.heapdict()
    state_queue[steps] = (heuristic_manhattan(init_state), init_state)

    while state_queue:
        steps, state = state_queue.popitem()
        state = state[1]
        states_visited.append(state)
        # print()
        # print("  ", state[0:3])
        # print("  ", state[3:6])
        # print("  ", state[6:9])

        if state == (1, 2, 3, 4, 5, 6, 7, 8, 0):
            break

        for move_id, move in enumerate(['r', 'l', 'u', 'd']):
            swap_pos = get_pos(state.index(0)) + np.array(moves[move])
            if np.all((swap_pos >= 0) & (swap_pos < 3)):  # Check that it is a valid move
                state_list = list(state)

                state_list[state.index(0)], state_list[get_index(swap_pos)] = state_list[get_index(swap_pos)], state_list[state.index(0)]
                new_state = tuple(state_list)

                if new_state not in states_visited:
                    state_queue[steps + move] = (len(steps) + heuristic(new_state), new_state)


    return steps, states_visited

In [ ]:
def test_astar(heuristic):
    test_cases = [
            ((1, 2, 3, 4, 5, 6, 7, 8, 0), "", [(1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 5, 6, 0, 7, 8), "rr", [(1, 2, 3, 4, 5, 6, 0, 7, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 6, 0, 7, 5, 8), "ldr", [(1, 2, 3, 4, 6, 0, 7, 5, 8),
                                                (1, 2, 3, 4, 0, 6, 7, 5, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 3, 0, 4, 2, 5, 7, 8, 6), "ldrd", [(1, 3, 0, 4, 2, 5, 7, 8, 6),
                                                (1, 0, 3, 4, 2, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 0, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 0, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
        ]

    for test in test_cases:
        state, actual_path, actual_states = test[0], test[1], test[2]
        path, states = astar(test[0], heuristic)
        assert path == actual_path and states == actual_states, f"Тест для расположения {state} завершился ошибкой: " \
                                                                f"astar вернула {path}, {states}, " \
                                                                    f"ожидаемые значения {actual_path, actual_states}"
    print("Все тесты для test_astar завершились успешно!")

test_astar(heuristic_misplaced)

Все тесты для test_astar завершились успешно!


In [ ]:
import random


def check_heuristics_difference():
    lengths_misplaced = []
    lengths_manhattan = []

    test_id = 0
    while True:
        if test_id >= 10:
            break

        start_state = list(range(9))
        random.shuffle(start_state)
        start_state = tuple(start_state)

        res_misplaced = astar(start_state, heuristic_misplaced)
        res_manhattan = astar(start_state, heuristic_manhattan)

        if res_misplaced[0] is not None:
            lengths_misplaced.append(len(res_misplaced[1]))
            lengths_manhattan.append(len(res_manhattan[1]))
            print(f"Тест {test_id + 1}:", len(res_misplaced[1]), len(res_manhattan[1]))
            test_id += 1

    print()
    print("Среднее по эвристике количества неправильно расположенных", sum(lengths_misplaced) / len(lengths_misplaced))
    print("Среднее по эвристике манхеттенских расстояний", sum(lengths_manhattan) / len(lengths_manhattan))
    return

In [ ]:
check_heuristics_difference()

Тест 0: 13980 852
Тест 1: 19444 1242
Тест 2: 4050 123
Тест 3: 5071 274
Тест 4: 5910 278
Тест 5: 4938 756
Тест 6: 16553 2553
Тест 7: 1810 213
Тест 8: 29740 1212
Тест 9: 56149 1624

Среднее по эвристике количества неправильно расположенных 15764.5
Среднее по эвристике манхеттенских расстояний 912.7


**Ввиду доминантности эвристики $L^1$-нормы над более простой эвристикой вида количества позиций, она показывает и более быструю сходимость на практике. Так и есть, манхеттенское расстояние более детально оценивает значение эвристики в текущем состоянии.**